# Projeto 2 - Ciência dos Dados

Nome: André Luís Silva Lopes

Nome: João Pedro Gianfaldoni Andrade

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [71]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [72]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
from nltk.corpus import stopwords
import numpy as np
import random
stopWords = set(stopwords.words('portuguese'))

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [73]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @JoaoAnd93050623

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [74]:
#Produto escolhido:
produto = 'SPTrans'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

### Capturando os dados do twitter:

In [75]:
#Cria um objeto para a captura
api = tweepy.API(auth)


#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text) and msg.author.screen_name != "sptrans": 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

### Salvando os dados em uma planilha Excel:

In [76]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()


### Classificando as mensagens na coragem

#### Os tweets foram classificados em duas categorias: Relevantes e Irrelevantes.
#### Foram considerados relevantes os tweets que criticavam a SPTrans, enquanto os irrelevantes eram todos os outros tweets que não os relevantes.

### Criando um dataframe a partir da tabela já classificada e separando em três dataframes, um dos relevantes , um dos irrelevantes, e um dos testes

In [262]:
SPTrans = pd.read_excel('SPTrans.xlsx')
SPTrans_teste = pd.read_excel('SPTrans.xlsx', "Teste")
SPTrans_I = SPTrans[SPTrans.Classificacao == "I"]
SPTrans_R = SPTrans[SPTrans.Classificacao == "R"]
SPTrans_R = SPTrans_R.reset_index().drop(columns = ["index"])
SPTrans_I = SPTrans_I.reset_index().drop(columns = ["index"])
Irrelevantes_raw = []
Relevantes_raw = []

### Criando duas listas, uma com todos tweets relevantes e uma com todos tweets irrelevantes

In [78]:
for i in range(0, len(SPTrans_R.Treinamento)):
    Relevantes_raw.append(SPTrans_R.Treinamento[i])
for i in range(0, len(SPTrans_I.Treinamento)):
    Irrelevantes_raw.append(SPTrans_I.Treinamento[i])

### Função que limpa os tweets

In [280]:
import re 
def cleanup(text):
    
    import string
    punctuation = '[!-.:?;]' 
    pattern = re.compile(punctuation)
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    #text = ' '.join(word for word in text.split() if word.lower() not in stopWords)   (tirar stopwords diminuiu a performance do modelo)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed  

###  Transformando as listas em strings limpas

In [80]:
Relevantes_raw_txt = ', '.join(Relevantes_raw)
Irrelevantes_raw_txt = ','.join(str(v) for v in Irrelevantes_raw)
Relevantes = cleanup(Relevantes_raw_txt.lower())
Irrelevantes = cleanup(Irrelevantes_raw_txt.lower())

### Criando uma serie para as palavras relevantes, uma para as palavras irrelevantes e uma com todas palavras

In [201]:
Serie_Relevantes  = pd.Series(Relevantes.split())
Serie_Irrelevantes  = pd.Series(Irrelevantes.split())
Tabela_Irrelevantes = Serie_Irrelevantes.value_counts()
Tabela_Relevantes = Serie_Relevantes.value_counts()
Palavras = Relevantes + Irrelevantes
Serie_Palavras = pd.Series(Palavras.split())
Tabela_Palavras = Serie_Palavras.value_counts(True)

___
### Montando o Classificador Naive-Bayes e verificando a performance

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Valores de Alpha e V utilizados no Laplace Smoothing, sendo V um valor "estimado"

In [319]:
alpha = 1
v = 950000

### Rodando o classificador nos dados de teste e verificando a performance e a matriz de confusão

In [320]:
P_Relevante = 1
P_Irrelevante = 1
A = 0
T = 0
TT = 0
FF = 0
TF = 0
FT = 0
for i in range (0, len(SPTrans_teste.Teste)):
    P_Relevante = 1
    P_Irrelevante = 1
    Tweet = SPTrans_teste.Teste[i]
    Tweet = cleanup(Tweet.lower())
    for palavra in Tweet.split():
        if palavra in Tabela_Relevantes:
            P_Relevante += np.log(((Tabela_Relevantes[palavra])+alpha)/(len(Tabela_Relevantes)+ alpha * v))
        else:
            P_Relevante += np.log(alpha/(len(Tabela_Relevantes)+ alpha * v))
        if palavra in Tabela_Irrelevantes:
            P_Irrelevante += np.log(((Tabela_Irrelevantes[palavra])+alpha)/(len(Tabela_Irrelevantes)+ alpha * v))
        else:
            P_Irrelevante += np.log(alpha/(len(Tabela_Irrelevantes)+ alpha * v))
    if P_Relevante > P_Irrelevante and SPTrans_teste.Classificacao[i] == "R":
        A+=1
        TT+=1
    elif P_Irrelevante > P_Relevante and SPTrans_teste.Classificacao[i] == "I":
        A+=1
        FF+=1
    elif P_Irrelevante < P_Relevante and SPTrans_teste.Classificacao[i] == "I":
        FT+=1
    elif P_Relevante < P_Irrelevante and SPTrans_teste.Classificacao[i] == "R":
        TF+=1

    T+=1
    
print("Número de acertos = ", (A))
print("Número de testes = ", (T))
print("Número de True-True = ", (TT))
print("Número de False-False = ", (FF))
print("Número de False-True = ", (FT))
print("Número de True-False = ", (TF))
print("Acurácia = ", (A/T))

Número de acertos =  130
Número de testes =  200
Número de True-True =  32
Número de False-False =  98
Número de False-True =  33
Número de True-False =  37
Acurácia =  0.65


### Testando o classificador com os mesmos dados utilizando a biblioteca "SKLearn"

In [321]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
S = SPTrans.dropna(axis=0)
Xt_train = S['Treinamento']
y_train = S['Classificacao'] == 'R'
count = CountVectorizer()
X_train = count.fit_transform(Xt_train)

model = MultinomialNB(alpha=0.5)
model.fit(X_train, y_train)
S_teste = pd.read_excel('SPTrans.xlsx', "Teste")
Xt_test = S_teste['Teste']
y_test = S_teste['Classificacao'] == 'R'
X_test = count.transform(Xt_test)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.675

___
### Concluindo

## Aperfeiçoamentos e perguntas:

### 1) Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento

#### Não seria ideal utilizar o classificador construído neste projeto, pois  ele apresenta acurácia não perfeita. Ou seja, a acertividade do modelo não é muito elevada, assim prejudicando possíveis amostras geradas pelo modelo. Neste caso observariamos uma propagação do erro identificado no classificador para as amostras prejudicando os dados gerados e assim criando um ciclo que cada vez mais amplia o erro do classificador.

### 2) Propor diferentes cenários para Naïve Bayes fora do contesto do projeto:

#### O Teorema de Naïve Bayes é frequentemente aplicado em processamento de linguagem natural (Aplicação explorada neste projeto) e diagnósticos médicos. Essa metodologia pode ser usada quando os atributos que descrevem as variáveis forem condicionalmente independentes. 
#### Dessa forma, um problema simples e comum que exemplifica bem o teorema é o cálculo de probabilidades em cima de diagnóstico de doenças. Explorando validação de testes laboratoriais e aferição de resultados. Podendo assim, probabilísticamente, confirmar ou negar a ocorrência da doença em um determinado paciênte. Ademais, o algoritmo é muito robusto para previsões em tempo real, ainda mais por precisar de poucos dados para realizar a classificação. Entretanto, caso haja necessidade de correlacionar fatores, como do tipo causa e consequência, o Naive Bayes tende a falhar na predição. Neste caso tornando-se uma ferramenta não muito útil.


### 3)Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)

#### Inicialmente, poderiam ser utilizados mais tweets na etapa de treinamento, o que aumentaria o "vocabulário" do modelo, e tornaria mais preciso a probabilidade de cada palavra pertencer ao grupo relevante ou irrelevante. Além disso, parte da amostra de treinamento poderia ser utilizada como validação, permitindo o ajuste de parametros como os paramentros utilizados no Laplace Smoothing sem gerar overfitting ou underfitting. Além disso as palavras poderiam ser "tratadas" utilizando técnicas como "Stemming" e "Lemmatization" (https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)


## Conclusão:

#### Após todo andamento do projeto e sua finalização, podemos concluir algumas coisas. Primeiramente fica evidente que o modelo foi bem sucedido dentro de um espaço limitado. Em outras palavras, o modelo se mostrou eficiente dentro do espaço amostral escolhido, demonstrando uma acurácia de 65%. Concluímos que este número é aceitável, visto que utilizando uma biblioteca para Python (Sklearn) que realiza esse processo de forma mais bem estruturada; chegamos a um valor de acertividade de 66%. Assim fica validado o classificador obtido ao fim deste projeto.
#### Podemos concluir também que provavelmente, para uma melhora na acurácia do modelo, seria interessante aumentar o número de tweets utilizados no treinamento. Desta forma o modelo teria contato com uma variedade ainda maior de construções de frases diferentes. Podendo assim ampliar "o repertório" alinhando melhor suas interpretações, visto que tratamos de uma aplicação de interpretação subjetiva (Linguagem Natural). Assim, com cerca de 10 mil tweets, exemplo, provável que a acurácia iria aumentar deixando o classificador mais efetivo. Podendo chegar à um classificador, "praticamente" ideal ("Quanto mais tweets melhor"). Dado que ele teve maior contato com diferentes "subjetividades" de escrita. 
 

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**